In [ ]:
# Code for ETL operations on Country-GDP data

# Importing the required libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime

In [97]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    time_format = '%Y-%h-%d:%H-%M-%S'
    current_time = datetime.now()
    timestamp = current_time.strfttime(time_format)
    with open('code_log.txt', 'a') as f:
        f.write(timestamp + ' : '+ str(message))
        
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    
    webpage = requests.get(url).text
    soup = BeautifulSoup(webpage,'html.parser')
    table = soup.find('table')
    
    df = pd.DataFrame(columns=table_attribs)

    for col in table.find_all('tr'):
        if len(col.find_all('td')) == 0:
            continue
        bank_dict = {table_attribs[1]: col.find_all('td')[1].get_text().strip(),
                     table_attribs[2]: float(col.find_all('td')[2].get_text().strip())}
        bank_df = pd.DataFrame(bank_dict, index=[0])
        df = pd.concat([df,bank_df], ignore_index=True) 
    return df

def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    exchange_rate = pd.read_csv(csv_path)
    exchange_rate_dict = exchange_rate.set_index('Currency').to_dict()['Rate']
    currency = ['MC_EUR_Billion' , 'MC_GBP_Billion', 'MC_INR_Billion']

    for i,j in zip(MC,exchange_rate_dict.keys()):
        df[i] = np.round(df['MC_USD_Billion'] * exchange_rate_dict[j],2)

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path, index=False)

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection,if_exist = 'replace', index= False)

def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    pd.read_sql(query_statement, sql_connection)

In [98]:
webpage = requests.get(url).text
soup = BeautifulSoup(webpage,'html.parser')
table = soup.find('table')


In [108]:
table.find_all('tr')[1].find_all('td')[1].find_all('a')[1]['title']

'JPMorgan Chase'